# Exploration tool for assessment schemes in ICNIRP 2020 and RPS S-1
author: Dr Vitas Anderson (*Two Fields Consulting*)

date: 20/7/2021

To use this Jupyter notebook, run each code cell in succession by either pressing the triangular play button ► in the menu bar above or by pressing SHIFT-ENTER for successive code cells. You can play around with the code as much as you like - it won't affect the original notebook.

Under the **Interactive Assessment Tool** heading below you will find an interactive tool for visualising how my assessment functions (under the **Functions** heading) work for two RF sources.

The function below is used to calculate test distributions for the limit normalised whole body point spatial levels over a vertical column of z points:

$\Large L_{wbps} = \frac{k_1}{\textrm{cosh}(k_2(z-z_{source}))}$

## Setup

In [1]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from collections import namedtuple
from collections.abc import Iterable
pd.options.display.max_rows = 201
np.set_printoptions(precision=4)

## Functions

In [2]:
def make_list(obj):
    '''make sure obj is a list'''
    if isinstance(obj, Iterable):
        return list(obj)
    else:
        return [obj]    

def check_source_inputs(*sources, n=None):
    '''Make sure that all source inputs are lists which are of the same length
       and are of length n if n is specified'''
    sources = [make_list(source) for source in sources]
    len_sources = [len(source) for source in sources]
    errmsg = f'All source inputs {sources} must have the same number of elements'
    assert len(set(len_sources)) == 1, errmsg
    if n != None:
        errmsg = f'All k1s, k2s, zs source inputs {sources} must each have {n} elements'
        assert len_sources[0] == n, errmsg
    
    return sources

def check_dz(dz,h,zlow,zhigh):
    '''Check that dz is a valid input'''
    # check that the z range is divisible by dz
    zrange = zhigh - zlow
    nz = int(zrange/dz)   # number of dz intervals in z range
    assert nz == zrange/dz, f"z range ({zrange}) must be divisible by dz ({dz})"
    
    # check that spatial average window length is divisible by dz
    nh = int(h/dz)    # number of dz intervals within spatial averaging window    
    nh2 = int(nh/2)   # number of dz intervals between the centre and the end of the spatial averaging window
    assert nh == h/dz, f"h ({h}) must be divisible by dz ({dz})"  

    return nz, nh, nh2
    
def calc_z(zlow, zhigh, nz):
    '''Calculate the z distribution'''
    z = np.linspace(zlow, zhigh, nz+1)
    return z

def calc_Lwbps(k1s,k2s,zs,z,nz):
    '''Calculate the whole body point spatial distribution for limit normalised exposure (Lwbps)'''
    Lwbps = np.zeros(nz+1)                  # initialise Lwbps array with zeros
    for k1i, k2i, zi in zip(k1s, k2s, zs):  # loop through each source
        Lwbps += k1i / np.cosh(k2i*(z-zi))    # add limit normalized field level for each source
    return Lwbps

def calc_Llocps(Lwbps, fMHz):
    '''Calculate the local point spatial distribution for limit normalised exposure (Llocps)'''
    # Calculate the local limit multiplier factor
    if fMHz <= 400:
        m = 5
    elif fMHz < 2000:
        m = 11.47459891 * fMHz**-0.138646884
    else:
        m = 4
        
    # Calculate and return Llocps
    return Lwbps / m

def calc_valid_nsap(nh, n=None):
    '''Return list of no more than n valid numbers of spatial averaging points
       for any value of nh, the number of dz intervals in h
       If n = None, then show all valid numbers of spatial averaging points'''
    valid_nsap = [i+1 for i in range(1, nh+1) if (nh % i == 0)]
    if (n != None) and (len(valid_nsap) > n):
        valid_nsap = valid_nsap[:n]
    return valid_nsap

def calc_hindices(nh, nh2, nsap):
    '''Determine the array indices of the spatial averaging points
       relative to the assessment point'''
    dh = int(nh / (nsap-1))  # number of indices between spatial averaging points
    hindices = [-nh2 + i*dh for i in range(nsap)]
    hindices = np.array(hindices, dtype=int)  # convert list to numpy integer array
    return hindices

def calc_Lwbsa(Lwbps, nz, nh2, hindices):
    '''Calculate the whole body spatial average distribution (Lwbsa)'''   
    # initialise Lwbsa array with NaN's (Not a Number)
    Lwbsa = np.repeat(np.nan, nz+1)
    
    # calculate the mean value of Lwbps at the spatial averaging points
    for iz in range(nh2, nz-nh2+1):
        Lwbsa[iz] = Lwbps[hindices + iz].mean()
        
    return Lwbsa
    
def calc_Lwbpssa(Lwbsa, nz, nh, nh2):
    '''Calculate the point spatial representation of the wb spatial average (Lwbpssa)
    INPUTS:
       Lwbsa = spatial average of the whole body limit normalised point spatial field over a vertical line
       nz = number of dz intervals in the z range
       nh = number of dz intervals in the h averaging window
       nh2 = number of dz intervals in half of the h averaging window
    OUTPUTS:
       Lwbpssa = point spatial respresentation of the spatially averaged field (Lwbsa)
       '''
    # initialise Lwbpssa array with NaN's (Not a Number)
    Lwbpssa = np.repeat(np.nan, nz+1)
    
    # first pass: assign Lwbpssa values based on minimum value of points that
    # are half an averaging window above or below the assessment point
    for iz in range(nh, nz-nh+1):
        Lwbpssa[iz] = Lwbsa[[iz-nh2, iz+nh2]].min() 
    
    # second pass: assign Lwbsa value at the point(s) where Lwbsa peaks,
    # i.e. where spatial averaging window exposure changes between head and feet
    ipeaks, _ = find_peaks(Lwbsa)  # get indices of local peaks of Lwbsa values    
    for iz in ipeaks:
        Lwbpssa[iz] = Lwbsa[iz]
        
    # third pass: assign minimum Lwbsa value for the length of the spatial averaging window
    # where Lwbsa is a minimum
    itroughs, _ = find_peaks(1/Lwbsa)  # get indices of local minimums of Lwbsa values
    for iz in itroughs:
        Lwbpssa[(iz-nh2):(iz+nh2)] = Lwbsa[iz]
        
    return Lwbpssa  

In [3]:
def calc_ass(h,nsap,fMHz,zlow,zhigh,dz,k1s,k2s,zs):
    '''Function which calculates assessment distributions over a set z height range for:
       + point spatial limit normalised field values for whole body exposure (Lwbps)
       + point spatial limit normalised field values for local exposure (Llocps)
       + spatial average of the whole body Lwbps points (Lwbsa) over a vertical line
       + point spatial representation of the whole body spatial averages (Lwbpssa)
       
       FUNCTION INPUTS:       
          h = height of the spatial averaging window (m)
       nsap = number of spatial averaging points [3 or 5]
       fMHz = exposure frequency in MHz
       zlow = lower bound of the z points (m)
      zhigh = upper bound of the z points (m)
        k1s = list of k1 parameters that set the peak level of the Lps distribution, 
              e.g. 2 for a single source or [2,1] for two sources
        k2s = list of k2 parameters that set the vertical beamwidth of the Lps distribution
              e.g. 1 for a single source or [1,2] for two sources
        zs = list of heights (m) for the source(s), 
              e.g. 7 for a single source, or [10,13] for two sources
         
       FUNCTION OUTPUTS:
         a named tuple, L, containing:
           + title: an informative title for the assessment
           + fMHz: exposure frequency in MHz
           + labels: lables for each calculated distribution
           + z: the z distribution points for the assessment
           + zlow: the lower bound of the z distribution
           + zhigh: the upper bound of the z distribution
           + lass: a named tuple for the calculated assessments (Lwbps, Llocps, Lwbsa, Lwbpssa)
           + nh: number of points in teh vertical averaging window
           + df: a pandas dataframe for z and Lwbps, Llocps, Lwbsa, Lwbpssa
      '''     
    
    # Check that all source inputs are lists which are of the same length
    k1s,k2s,zs = check_source_inputs(k1s,k2s,zs)
    
    # Check that the dz value is valid for h and the z range
    nz, nh, nh2 = check_dz(dz,h,zlow,zhigh)
    
    # Check that the nsap input is valid for the number of h points
    valid_nsap = calc_valid_nsap(nh)
    assert nsap in valid_nsap, f'Value of nsap ({nsap}) must be one of {valid_nsap}'
    
    # Calculate the z points
    z = calc_z(zlow,zhigh,nz)
    
    # Calculate the whole body point spatial distribution for limit normalised exposure (Lwbps)
    Lwbps = calc_Lwbps(k1s,k2s,zs,z,nz)
    
    # Calculate the local point spatial distribution for limit normalised exposure (Llocps)
    Llocps = calc_Llocps(Lwbps, fMHz)
    
    # Calculate the whole body spatial average distribution (Lwbsa)
    hindices = calc_hindices(nh, nh2, nsap)
    Lwbsa = calc_Lwbsa(Lwbps, nz, nh2, hindices)
    
    # Calculate the point spatial representation of the wb spatial average (Lwbpssa)
    Lwbpssa = calc_Lwbpssa(Lwbsa, nz, nh, nh2)
            
    ## Create a pandas dataframe of z and all assessment distributions
    df = pd.DataFrame(dict(z=z,Lwbps=Lwbps,Llocps=Llocps,Lwbsa=Lwbsa,Lwbpssa=Lwbpssa))
    
    ## Create labels for the limit normalised assessment distributions
    labels = ['wb point spatial',
              'local point spatial',
              'wb spatial average',
              'wb point spatial spat. avg.']
    
    ## Create a title for the data set
    title = f'{nsap} points over {h}m\n' + '$k_1$' + f'={",".join(map(str,k1s))}, ' + '$k_2$' + f'={",".join(map(str,k2s))}, f={fMHz} MHz'
    
    ## Create a named tuple for all the L assessment distributions
    Lass = namedtuple("Lass", "Lwbps, Llocps, Lwbsa, Lwbpssa")
    lass = Lass(Lwbps, Llocps, Lwbsa, Lwbpssa)
    
    ## Create a named tuple for all the output data
    Ldata = namedtuple("Ldata", "title fMHz labels z zlow zhigh lass nh df")
    ldata = Ldata(title, fMHz, labels, z, zlow, zhigh, lass, nh, df)

    return ldata

def plotL(L):
    '''Plot the L distributions'''
    fig, ax = plt.subplots(figsize=(4,8))
    for lass, label in zip(L.lass, L.labels):
        ax.plot(lass, L.z, alpha=0.6, label=label)
    ax.set_xlabel('Limit normalised field value')
    ax.set_ylabel('z (m)')
    ax.grid(ls='--')
    ax.legend()
    zmin, zmax = int(L.zlow), int(L.zhigh)
    ax.set_yticks(range(zmin,zmax))
    ax.set_ylim(zmin,zmax)
    ax.set_title(L.title)
    fig.tight_layout()
    
    return ax

## Object class for interactive tool

In [4]:
plt.close('all')
def make_box_layout():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

def make_box_layout2():
     return widgets.Layout(
        border='solid 1px black',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px'
     )

class Ass(widgets.HBox):
    '''This object class generates an interactive widget tool for calculating
       assessment distributions over a set z height range for:
       + point spatial limit normalised field values for whole body exposure (Lwbps)
       + point spatial limit normalised field values for local exposure (Llocps)
       + spatial average of the whole body Lps points (Lwbsa) over a vertical line
       + point spatial representation of the whole body spatial averages (Lwbpssa)
       
       FUNCTION INPUTS:       
          h = height of the spatial averaging window (m)
       nsap = number of spatial averaging points [3 or 5]
   nsap_num = maximum umber of valid nsap options to display in the nsap radio widget
       fMHz = exposure frequency in MHz
       zlow = lower bound of the z points (m)
      zhigh = upper bound of the z points (m)
         dz = distance increment between z points (m)
        k1s = list of k1 parameters that set the peak level of the Lps distribution, 
              e.g. 2 for a single source or [2,1] for two sources
        k2s = list of k2 parameters that set the vertical beamwidth of the Lps distribution
              e.g. 1 for a single source or [1,2] for two sources
        zs = list of heights (m) for the source(s), 
              e.g. 7 for a single source, or [10,13] for two sources
        plot_height = height of the plot in the widget tool (in)
         
       FUNCTION OUTPUTS:
         an interactive widget tool which can be displayed using: display(Ass_object)
      '''  
    
    def __init__(self, h=2, nsap=5, nsap_num=7, fMHz=400, 
                 zlow=0, zhigh=12, dz=0.1,
                 k1s=[1,2], k2s=[2,2], zs=[8, 4],
                 plot_height=8):
        super().__init__()
        
        # Create output widget for the plot
        output = widgets.Output()
        
        # Create self variables from init values
        self.h = h
        self.nsap = nsap
        self.nsap_num = nsap_num
        self.fMHz = fMHz
        self.zlow = zlow
        self.zhigh = zhigh
        self.dz = dz
        
        # Do input value checks
        self.k1s,self.k2s,self.zs = check_source_inputs(k1s,k2s,zs)
        self.nz, self.nh, self.nh2 = check_dz(dz,h,zlow,zhigh)
        
        # Calculate z distribution
        self.z = calc_z(self.zlow,self.zhigh,self.nz)
        
        # Calculate all initial assessment values
        self.nsap, self.valid_nsap, self.nh, self.nh2, self.hindices = \
          self.calc_variables(self.dz,self.h,self.zlow,self.zhigh,self.nsap,self.nsap_num)
        self.calc_ass()

        # Create plot
        with output:
            self.fig, self.ax = plt.subplots(constrained_layout=True, figsize=(4, plot_height))
            
        self.lineLwbps, = self.ax.plot(self.Lwbps, self.z, alpha=0.5, label='wb point spatial')
        self.lineLlocps, = self.ax.plot(self.Llocps, self.z, alpha=0.5, label='local point spatial')
        self.lineLwbsa, = self.ax.plot(self.Lwbsa, self.z, alpha=0.5, label='wb spatial average')
        self.lineLwbpssa, = self.ax.plot(self.Lwbpssa, self.z, alpha=0.5, label='wb point spatial spatial avg')
        
        self.fig.canvas.toolbar_position = 'bottom'
        self.fig.canvas.header_visible = False
        self.ax.set_xlabel('Limit normalised field value')
        self.ax.set_ylabel('z (m)')
        self.ax.grid(True, ls='--', alpha=0.5)
        self.ax.legend()
        zmin, zmax = int(self.zlow), int(self.zhigh)
        self.ax.set_yticks(range(zmin,zmax+1))
        self.ax.set_ylim(zmin,zmax)
        self.set_title()

        # Plot annotation 
        # Use this if needed for debugging. Update text with self.set_text
        self.text = self.ax.text(0.3,12,'')

        # define widgets
        zsA_slider = widgets.FloatSlider(
            value=self.zs[0],
            min=self.zlow, 
            max=self.zhigh, 
            step=self.dz, 
            description='Source A',
            orientation='vertical',
            layout=widgets.Layout(height=f'{plot_height}in'),
        )
        zsB_slider = widgets.FloatSlider(
            value=self.zs[1],
            min=self.zlow, 
            max=self.zhigh, 
            step=self.dz, 
            description='Source B',
            orientation='vertical',
            layout=widgets.Layout(height=f'{plot_height}in'),
        )
        self.nsap_radio = widgets.RadioButtons(
            options=self.valid_nsap,
            value=self.nsap,
            description='nsap:',
            orientation='horizontal',
            disabled=False
        )
        k1A_float = widgets.FloatText(
            value=self.k1s[0],
            step=0.1,
            description='k1',
            continuous_update=False,
            disabled=False,
        )               
        k2A_float = widgets.FloatText(
            value=self.k2s[0],
            step=0.1,
            description='k2',
            continuous_update=False,
            disabled=False,
        )               
        k1B_float = widgets.FloatText(
            value=self.k1s[1],
            step=0.1,
            description='k1',
            continuous_update=False,
            disabled=False,
        )               
        k2B_float = widgets.FloatText(
            value=self.k2s[1],
            step=0.1,
            description='k2',
            continuous_update=False,
            disabled=False,
        )               
        h_slider = widgets.FloatSlider(
            value=self.h,
            min=0.2, 
            max=2, 
            step=0.1, 
            description='h (m)',
            layout=widgets.Layout(width='4in'),
            readout_format='.1f',
            continuous_update=False,
        )
        
        # define left controls
        height_sliders = widgets.HBox([
            zsA_slider, 
            zsB_slider, 
        ])
        
        left_controls = widgets.VBox([
            widgets.HTML("<u>Source heights</u>"),
            height_sliders,            
        ])
        
        left_controls.layout = make_box_layout2()
        
        # define right controls
        right_controls = widgets.VBox([
            widgets.HTML("<u>Spatial averaging parameters</u>"),
            h_slider,
            self.nsap_radio,
            widgets.HTML("<u>Source A</u>"),
            k1A_float,
            k2A_float,
            widgets.HTML("<u>Source B</u>"),
            k1B_float, 
            k2B_float,
        ])
        
        right_controls.layout = make_box_layout()
        
        # define output
        out_box = widgets.Box([output])
        output.layout = make_box_layout()

        # observe stuff
        zsA_slider.observe(self.zsA_change, 'value')
        zsB_slider.observe(self.zsB_change, 'value')       
        h_slider.observe(self.h_change, 'value')       
        k1A_float.observe(self.k1A_change, 'value')       
        k2A_float.observe(self.k2A_change, 'value')       
        k1B_float.observe(self.k1B_change, 'value')       
        k2B_float.observe(self.k2B_change, 'value')
        self.nsap_radio.observe(self.nsap_change, 'value')

        # add to Ass object's children
        self.children = [left_controls, output, right_controls]
    
    def set_text(self):
        txt = f'valid_nsap = {self.valid_nsap}\nnsap = {self.nsap}\nh = {self.h}\nnh = {self.nh}'
        self.text.set_text(txt)
    
    def nearest_nsap(self, nsap, nh, nsap_num=None):
        '''Get the nearest valid value of nsap to the current nsap'''
        valid_nsap = calc_valid_nsap(nh, n=nsap_num)
        abs_diff = np.abs(np.array(valid_nsap) - nsap)
        index = abs_diff.argmin()
        nsap = valid_nsap[index]
        return nsap, valid_nsap

    def calc_variables_old(self):
        '''Calculate all preliminary assessment variables'''
        self.nz,self.nh,self.nh2 = check_dz(self.dz,self.h,self.zlow,self.zhigh)
        self.nsap,self.valid_nsap = self.nearest_nsap(self.nsap,self.nh,self.nsap_num)
        self.hindices = calc_hindices(self.nh, self.nh2, self.nsap)
        
    def calc_variables(self, dz, h, zlow, zhigh, nsap, nsap_num):
        '''Calculate preliminary assessment variables'''
        nz, nh, nh2 = check_dz(dz, h, zlow, zhigh)
        nsap, valid_nsap = self.nearest_nsap(nsap, nh, nsap_num)
        hindices = calc_hindices(nh, nh2, nsap)
        return nsap, valid_nsap, nh, nh2, hindices 
        
    def calc_ass(self):
        '''Calculate all assessment distributions'''
        self.Lwbps = calc_Lwbps(self.k1s,self.k2s,self.zs,self.z,self.nz)
        self.Llocps = calc_Llocps(self.Lwbps, self.fMHz)
        self.Lwbsa = calc_Lwbsa(self.Lwbps, self.nz, self.nh2, self.hindices)
        self.Lwbpssa = calc_Lwbpssa(self.Lwbsa, self.nz, self.nh, self.nh2)
           
    def set_title(self):
        title = f'{self.nsap} points over {self.h}m\n' + \
                '$k_1$' + f'={",".join(map(str,self.k1s))}, ' + \
                '$k_2$' + f'={",".join(map(str,self.k2s))}, f={self.fMHz} MHz'
        self.fig.suptitle(title)

    def line_update(self, Lwbps, Llocps, Lwbsa, Lwbpssa):
        """update assessment lines in plot"""
        self.lineLwbps.set_xdata(Lwbps)
        self.lineLlocps.set_xdata(Llocps)
        self.lineLwbsa.set_xdata(Lwbsa)
        self.lineLwbpssa.set_xdata(Lwbpssa)
        self.fig.canvas.draw()

    def h_change(self, change):
        self.h = change.new
        self.nsap, self.valid_nsap, self.nh, self.nh2, self.hindices = \
          self.calc_variables(self.dz,self.h,self.zlow,self.zhigh,self.nsap,self.nsap_num)
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)
        self.set_title()
        
        # Update nsap_radio widget. Note that we need to 'unobserve' the nsap_radio widget
        # while updating it since it will otherwise trigger a callback when changing it
        self.nsap_radio.unobserve(self.nsap_change, 'value')
        self.nsap_radio.options = self.valid_nsap
        self.nsap_radio.value = self.nsap
        self.nsap_radio.observe(self.nsap_change, 'value')
        
    def nsap_change(self, change):
        self.nsap = change.new
        self.nsap, self.valid_nsap, self.nh, self.nh2, self.hindices = \
          self.calc_variables(self.dz,self.h,self.zlow,self.zhigh,self.nsap,self.nsap_num)
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)
        self.set_title()
        
    def zsA_change(self, change):
        self.zs[0] = change.new
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)
        
    def zsB_change(self, change):
        self.zs[1] = change.new
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)
        
    def k1A_change(self, change):
        self.k1s[0] = change.new
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)   

    def k2A_change(self, change):
        self.k2s[0] = change.new
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)   

    def k1B_change(self, change):
        self.k1s[1] = change.new
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)   

    def k2B_change(self, change):
        self.k2s[1] = change.new
        self.calc_ass()
        self.line_update(self.Lwbps, self.Llocps, self.Lwbsa, self.Lwbpssa)   

## Interactive assessment tool
Use this tool to explore the effect of spatial averaging schemes on different exposure scenarios generated by two hypothetical RF sources over a vetical line of points in the z direction.

**Cumulative limit-normalised field level:**
$\Large L_{wbps} = \frac{k_1}{\textrm{cosh}(k_2(z-z_{source}))}$

In [5]:
def calc_Lwbpssa(Lwbsa, nz, nh, nh2):
    '''Calculate the point spatial representation of the wb spatial average (Lwbpssa)
    INPUTS:
       Lwbsa = spatial average of the whole body limit normalised point spatial field over a vertical line
       nz = number of dz intervals in the z range
       nh = number of dz intervals in the h averaging window
       nh2 = number of dz intervals in half of the h averaging window
    OUTPUTS:
       Lwbpssa = point spatial respresentation of the spatially averaged field (Lwbsa)
       '''
    # initialise Lwbpssa array with NaN's (Not a Number)
    Lwbpssa = np.repeat(np.nan, nz+1)
    
    # first pass: assign Lwbpssa values based on minimum value of points that
    # are half an averaging window above or below the assessment point
    for iz in range(nh, nz-nh+1):
        Lwbpssa[iz] = Lwbsa[[iz-nh2, iz+nh2]].min() 
    
    # second pass: assign Lwbsa value at the point(s) where Lwbsa peaks,
    # i.e. where spatial averaging window exposure changes between head and feet
    ipeaks, _ = find_peaks(Lwbsa)  # get indices of local peaks of Lwbsa values    
    for iz in ipeaks:
        Lwbpssa[iz] = Lwbsa[iz]
        
    # third pass: assign minimum Lwbsa value for the length of the spatial averaging window
    # where Lwbsa is a minimum
    itroughs, _ = find_peaks(1/Lwbsa)  # get indices of local minimums of Lwbsa values
    for iz in itroughs:
        Lwbpssa[(iz-nh2):(iz+nh2)] = Lwbsa[iz]
        
    return Lwbpssa  

In [6]:
# Create an isntance of the assessment explorer
# Alter z increment (dz), height of z points (zhigh), plot height (plot_height)
# and exposure frequency (fMHz) in the Ass arguments below

ass1 = Ass(dz=0.1, zhigh=16, plot_height=8, fMHz=400)
display(ass1)

Ass(children=(VBox(children=(HTML(value='<u>Source heights</u>'), HBox(children=(FloatSlider(value=8.0, descri…

## Explore different configurations

In [7]:
L1 = calc_ass(h=2, nsap=5, fMHz=400,
              zlow=0, zhigh=12, dz=0.01,
              k1s=[1,2], k2s=[2,2],
              zs=[4, 8])

plotL(L1);
plt.savefig("../plots/2 source 5 pts far peaks.png",dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
L1 = calc_ass(h=2, nsap=5, fMHz=400,
              zlow=0, zhigh=12, dz=0.01,
              k1s=[1,2], k2s=[2,2], zs=[4, 8]
             )

plotL(L1);
plt.savefig("../plots/2 source 5 pts far peaks.png",dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
L1 = calc_ass(h=2, nsap=5, fMHz=400,
              zlow=0, zhigh=12, dz=0.01,
              k1s=[1,2], k2s=[2,2], zs=[4, 6]
             )

plotL(L1);
plt.savefig("../plots/2 source 5 pts close peaks.png",dpi=100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
plt.gcf().savefig("../plots/2 source 5 pts.png",dpi=300)

In [9]:
plt.close('all')
L1 = calc_ass(h=2, nsap=3, fMHz=400,
              zlow=0, zhigh=12, dz=0.1,
              k1s=[1]*2, k2s=[2]*2, zs=[4, 8]
             )

plotL(L1);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Vary $k_1$ and $k_2$ 

In [10]:
fig, axes = plt.subplots(1,4,figsize=(9,8))
for j, (k1,k2) in enumerate(zip([1,1,1,2],[0.5,1,2,1])):
    
    # Calculate the assesments
    L = calc_ass(h=2, nsap=5, fMHz=400,
                 zlow=0, zhigh=20, dz=0.1,
                 k1s=k1, k2s=k2, zs=10
                )
    
    # Create the plots
    ax = axes[j]
    ax.plot(L.lass.Lwbps, L.z, alpha=0.7, label=L.labels[0])
    ax.set_xlabel('Limit normalised field value')
    ax.set_ylabel('z (m)')
    ax.grid(ls='--')
    ax.legend(fontsize=8,loc='upper right')
    zmin, zmax = int(L.zlow), int(L.zhigh)
    ax.set_yticks(range(zmin,zmax))
    ax.set_ylim(zmin,zmax)
    ax.set_title(L.title, fontsize=10)
    
fig.tight_layout(w_pad=2)
# fig.savefig('../plots/k1 k2 plots.png', dpi=100)
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Vary nsap and $k_2$ for a <u>single</u> source

In [11]:
fig, axes = plt.subplots(2,3,figsize=(9,12))
for i, nsap in enumerate([3,5]):
    for j, k2 in enumerate([2, 1, 0.5]):
        
        # Calculate the assessments
        L = calc_ass(h=2, nsap=nsap, fMHz=400,
                     zlow=0, zhigh=20, dz=0.01,
                     k1s=1, k2s=k2,
                     zs=10)
        
        # Create the plots 
        ax = axes[i,j]
        for lass, label in zip(L.lass, L.labels):
            ax.plot(lass, L.z, alpha=0.7, label=label)
        ax.set_xlabel('Limit normalised field value')
        ax.set_ylabel('z (m)')
        ax.grid(ls='--')
        ax.legend(fontsize=8,loc='upper right')
        zmin, zmax = int(L.zlow), int(L.zhigh)
        ax.set_yticks(range(zmin,zmax))
        ax.set_ylim(zmin,zmax)
        ax.set_title(L.title,fontsize=10)
        
    fig.tight_layout(h_pad=4,w_pad=2)

# fig.savefig('../plots/nsap k2 plots 1source.png', dpi=100)        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Vary nsap and $k_2$ for <u>two</u> sources

In [12]:
fig, axes = plt.subplots(2,3,figsize=(9,12))
for i, nsap in enumerate([3,5]):
    for j, k2 in enumerate([2, 1, 0.5]):

        # Calculate the assessments
        L = calc_ass(h=2, nsap=nsap, fMHz=400,
                     zlow=0, zhigh=20, dz=0.01,
                     k1s=[1]*2, k2s=[k2]*2,
                     zs=[7,13])
        
        # Create the plots
        ax = axes[i,j]
        for lass, label in zip(L.lass, L.labels):
            ax.plot(lass, L.z, alpha=0.7, label=label)
        ax.set_xlabel('Limit normalised field value')
        ax.set_ylabel('z (m)')
        ax.grid(ls='--')
        ax.legend(fontsize=8,loc='upper right')
        zmin, zmax = int(L.zlow), int(L.zhigh)
        ax.set_yticks(range(zmin,zmax))
        ax.set_ylim(zmin,zmax)
        ax.set_title(L.title)
    fig.tight_layout(h_pad=4,w_pad=2)

# fig.savefig('../plots/nsap k2 plots 2source.png', dpi=100)
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Scratch
*for testing out code ...*

### Test out assessment functions

In [14]:
# INPUTS
h=2
nsap=5
fMHz=400
zlow=0
zhigh=12
dz=0.1
k1s=[1,2]
k2s=[2,2]
zs=[4, 8]

# Calculation parameters
nz, nh, nh2 = check_dz(dz,h,zlow,zhigh)
valid_nsap = calc_valid_nsap(nh)      
hindices = calc_hindices(nh, nh2, nsap)
k1s,k2s,zs = check_source_inputs(k1s,k2s,zs)

print(f'nz={nz}, nh={nh}, nh2={nh2}')
print(f'valid nsp: {valid_nsap}')
print(f'hindices={hindices}')
print(f'k1s={k1s}, k2s={k2s}, zs={zs}')

# Calculated assessment distributions
z = calc_z(zlow,zhigh,nz)
Lwbps = calc_Lwbps(k1s,k2s,zs,z,nz)
Llocps = calc_Llocps(Lwbps, fMHz)
Lwbsa = calc_Lwbsa(Lwbps, nz, nh2, hindices)
Lwbpssa = calc_Lwbpssa(Lwbsa, nz, nh, nh2)

print('\nz\n',z)
print('\nLwbps\n', Lwbps)
print('\nLwbsa\n', Lwbsa)
print('\nLlocps\n', Llocps)
print('\nLwbpssa\n', Lwbpssa)

nz=120, nh=20, nh2=10
valid nsp: [2, 3, 5, 6, 11, 21]
hindices=[-10  -5   0   5  10]
k1s=[1, 2], k2s=[2, 2], zs=[4, 8]

z
 [ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1.   1.1  1.2  1.3
  1.4  1.5  1.6  1.7  1.8  1.9  2.   2.1  2.2  2.3  2.4  2.5  2.6  2.7
  2.8  2.9  3.   3.1  3.2  3.3  3.4  3.5  3.6  3.7  3.8  3.9  4.   4.1
  4.2  4.3  4.4  4.5  4.6  4.7  4.8  4.9  5.   5.1  5.2  5.3  5.4  5.5
  5.6  5.7  5.8  5.9  6.   6.1  6.2  6.3  6.4  6.5  6.6  6.7  6.8  6.9
  7.   7.1  7.2  7.3  7.4  7.5  7.6  7.7  7.8  7.9  8.   8.1  8.2  8.3
  8.4  8.5  8.6  8.7  8.8  8.9  9.   9.1  9.2  9.3  9.4  9.5  9.6  9.7
  9.8  9.9 10.  10.1 10.2 10.3 10.4 10.5 10.6 10.7 10.8 10.9 11.  11.1
 11.2 11.3 11.4 11.5 11.6 11.7 11.8 11.9 12. ]

Lwbps
 [6.7138e-04 8.2002e-04 1.0016e-03 1.2233e-03 1.4942e-03 1.8250e-03
 2.2290e-03 2.7226e-03 3.3253e-03 4.0616e-03 4.9608e-03 6.0591e-03
 7.4006e-03 9.0390e-03 1.1040e-02 1.3484e-02 1.6469e-02 2.0115e-02
 2.4567e-02 3.0005e-02 3.6644e-02 4.4749e-02 5.4643e-

In [6]:
df = pd.DataFrame(np.arange(12).reshape(4,3),columns=list('abc'))
df

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11


In [10]:
# mean absolute deviation function - very cool!
df.mad(axis=1)

0    0.666667
1    0.666667
2    0.666667
3    0.666667
dtype: float64

In [19]:
# transform applies functions on each single value
df.transform([np.sqrt,np.exp])

a                      b                       c              
       sqrt          exp      sqrt           exp      sqrt           exp
0  0.000000     1.000000  1.000000      2.718282  1.414214      7.389056
1  1.732051    20.085537  2.000000     54.598150  2.236068    148.413159
2  2.449490   403.428793  2.645751   1096.633158  2.828427   2980.957987
3  3.000000  8103.083928  3.162278  22026.465795  3.316625  59874.141715

In [31]:
# agg applies function on aggregated values, i.e. in columns or rows
df.agg(['mean','std','min','max'],axis=0)

,a,b,c
mean,4.500000,5.500000,6.500000
std,3.872983,3.872983,3.872983
min,0.000000,1.000000,2.000000
max,9.000000,10.000000,11.000000


In [30]:
# agg applies function on aggregated values, i.e. in columns or rows
df.agg({'a' : ['sum', 'min'], 'b' : ['min', 'max']})

,a,b
sum,18.0,NaN
min,0.0,1.0
max,NaN,10.0


In [32]:
df.apply({'a' : ['sum', 'min'], 'b' : ['min', 'max']})

,a,b
sum,18.0,NaN
min,0.0,1.0
max,NaN,10.0


In [3]:
a = 79
b = 2 if a == 7 else 4
print(a,b)

79 4
